This notebook is meant for the CLUSTER version of analysis, for the purposes of creating an entropy profile that mimics the Phoenix cluster and similar clusters that lack an AGN that heats against the cooling flow.

In [ ]:
import random
import statistics
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import odeint
from scipy.stats import linregress
from scipy.interpolate import interp1d
from scipy.integrate import solve_ivp
from scipy.constants import k as k_B  # Boltzmann constant in J/K
from scipy.constants import G, proton_mass
from functools import partial
# from lab_functions_1 import Ax, vcgrab, Tc, Lambdacalc, dLdTfunc, dVcdrfunc
import lab_functions_1 as lf
mp_g = proton_mass*1e3
k_Bcgs = k_B*1e7
mu = 0.6
gamma = 5/3
ktc = 3.0857e21
etkv = 6.2415*10**8

In [ ]:
from scipy.integrate import cumulative_trapezoid
ktc = 3.0857e21
r_grid = np.linspace(ktc, 1000000 * ktc, 1000000)  # finer grid = better accuracy
vc2_over_r = np.array([(lf.vcgrab(r, 620 * ktc, 2060 * ktc, 5e10, 1e15)**2 / r) for r in r_grid])

phi_cumint = cumulative_trapezoid(vc2_over_r, r_grid, initial=0.0)

phi_values = -(phi_cumint[-1] - phi_cumint)

phi_interp = interp1d(r_grid, phi_values, kind='cubic', fill_value="extrapolate")

def phi(r):
    """Interpolated gravitational potential at any r."""
    return phi_interp(r)

In [ ]:
r_gridvc = np.logspace(np.log10(ktc), np.log10(1e25), 10000)  # finer grid = better accuracy
vc2_over_r = np.array([lf.vcgrab(r, 620 * ktc, 2060 * ktc, 5e10, 1e15) for r in r_gridvc])

plt.scatter(r_gridvc, vc2_over_r, s=1)
plt.xlabel('radius (r)')
plt.xlim(3.086e21, 3.086e24)
plt.xscale('log')
plt.ylabel('circular velocity (v_c)')
plt.ylim(1e6, 3e8)
plt.yscale('log')
plt.title('circular velocity across radii')
plt.grid(True, which='both', linestyle='--', alpha=0.7)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
def Rsonfinder(tol, r_high, r_low, max_iter=1000):
    iter_count = 0
    target = 1.89e28
    delta = tol * 1e27

    while r_high - r_low > tol:
        if iter_count > max_iter:
            print("Max iterations reached without convergence.")
            return None, None

        r = 0.5 * (r_high + r_low)
        print(f"Trying r = {r:.5e}")

        cs2_sonic = lf.vcgrab(r, 620 * ktc, 2060 * ktc, 5e10, 1e15)**2 / (2 * 0.7)
        v_sonic = cs2_sonic**0.5
        T_sonic = mu * mp_g * cs2_sonic / (gamma * k_Bcgs)
        tflow_to_tcool = 1

        rho_sonic = lf.rhocalc(v_sonic, tflow_to_tcool, T_sonic, r)
        Mdot = 4 * np.pi * r**2 * rho_sonic * v_sonic

        if Mdot > target + delta:
            print(f"Mdot = {Mdot:.2e} too high, decreasing r")
            r_high = r*1.1
        elif Mdot < target - delta:
            print(f"Mdot = {Mdot:.2e} too low, increasing r")
            r_low = r*0.9
        else:
            print(f"Converged: Mdot = {Mdot:.2e}, r = {r:.5e}")
            return r, Mdot

        iter_count += 1

    print("Did not converge within tolerance.")
    return None, None

In [ ]:
def dlnTdlnrcalc(R_sonic, x, T_sonic_point,pr=True): 
 
    dlnLambda_dlnT = lf.Lambdacalc(T_sonic_point)
    dlnvc_dlnR = lf.dVcdrfunc(R_sonic, 27.1*ktc, 210*ktc, 6.3e10, 1e12)
    
    #solve quadratic equation    
    b = 29/6.*x - 17/6. + 1/3.*(1.-x)*(dlnLambda_dlnT)
    c = 2/3.*x*dlnvc_dlnR + 5*x**2 - 13/3.*x + 2/3.
    if b**2-4*c >= 0:
        return [(-b +j * (b**2-4*c)**0.5)/2. for j in (-1,1)]
    else:
        if pr: print('no transsonic solutions')
        return None, None

In [ ]:
# THE ODE SOLVER ITSELF
def TheODE(r, C, Mdot):
    v, T = C
    T = np.clip(T, 1.585e4, 1.445e8)
    
    vc2 = lf.vcgrab(r, 27.1*ktc, 210*ktc, 6.3e10, 1e12)**2
    cs2 = (gamma * k_Bcgs * T) / (mu * mp_g)
    
    tflow = r / np.abs(v)
    mach = v / np.sqrt(cs2)
    
    Lambda = lf.Lambdacalc(T)
    rho = Mdot / (4 * np.pi * r**2 * v)
    n = rho / (mu * mp_g)
    
    tcool = (3 * k_Bcgs * T) / (2 * Lambda * n)

    dlnvdlnr = (2 - (vc2 / cs2) - (tflow/ (gamma*tcool))) / (mach**2 - 1.0)
    dlnTdlnr = (tflow / tcool) - (2/3) * (2 + dlnvdlnr)
    
    dvdr = (v / r) * dlnvdlnr
    dTdr = (T / r) * dlnTdlnr
    
    # if mach > 1.0:
    #     print(f"WARNING: Mach = {mach:.3f} at r = {r:.3e} cm → is supersonic!")
    return [dvdr, dTdr]

In [ ]:
vc_csa = []
tflow_tcoola = []
mach2a = []
dlnvdlnra = []
dlnTdlnra = []

ra2 = []

varray = []
Tarray = []
rhoarray = []


def TheODETracker(r, C, Mdot):
    v, T = C
    T = np.clip(T, 1.585e4, 1.445e8)
    
    vc2 = lf.vcgrab(r, 27.1*ktc, 210*ktc, 6.3e10, 1e12)**2
    cs2 = (gamma * k_Bcgs * T) / (mu * mp_g)
    
    tflow = r / np.abs(v)
    mach = v / np.sqrt(cs2)
    
    Lambda = lf.Lambdacalc(T)
    rho = Mdot / (4 * np.pi * r**2 * v)
    n = rho / (mu * mp_g)
    
    tcool = (3 * k_Bcgs * T) / (2 * Lambda * n)

    dlnvdlnr = (2 - (vc2 / cs2) - (tflow/ (gamma*tcool))) / (mach**2 - 1.0)
    dlnTdlnr = (tflow / tcool) - (2/3) * (2 + dlnvdlnr)
    
    dvdr = (v / r) * dlnvdlnr
    dTdr = (T / r) * dlnTdlnr

    vc_csa.append(2 - (vc2 / cs2))
    tflow_tcoola.append(tflow / tcool)
    mach2a.append(mach**2 - 1.0)
    dlnvdlnra.append(dlnvdlnr)
    dlnTdlnra.append(dlnTdlnr)
    ra2.append(r)
    varray.append(v)
    Tarray.append(T)
    rhoarray.append(rho)
    return [dvdr, dTdr]

In [ ]:
# CLASSES (THE BANE OF MY EXISTENCE)
class IntegrationResult:
    def __init__(self, res, stop_reason):
        self.res = res
        self._stop_reason = stop_reason
    
    def stopReason(self):
        return self._stop_reason
    
    def Rs(self):
        return self.res.t
    
    def __getitem__(self, key):
        return self.res[key]


# EVENTS LIST
def event_unbound(r, C, Mdot):
    v, T = C
    cs2 = (gamma * k_Bcgs * T) / (mu * mp_g)
    phi_r = phi(r)
    bern = 0.5 * v**2 + 1.5 * cs2 + phi_r
    return bern
event_unbound.terminal = True
event_unbound.direction = 1

def event_lowT(r, C, Mdot):
    T = C[1]
    return T - (10**4.2)
event_lowT.terminal = True
event_lowT.direction = -1 

def event_sonic_point(r, C, Mdot):
    v, T = C
    cs2 = (gamma * k_Bcgs * T) / (mu * mp_g)
    mach = v / np.sqrt(cs2)
    return mach - 1.0
event_sonic_point.terminal = True
event_sonic_point.direction = -1

def event_max_R(r, C, Mdot):
    v, T = C
    return r - (20600*ktc)
event_max_R.terminal = True
event_max_R.direction = 1

my_event_list = [
    event_sonic_point,
    event_unbound,
    event_lowT,
    event_max_R
]


event_names = ['sonic point', 'unbound', 'lowT', 'max R reached']

# SHOOTING METHOD
def sonic_point_shooting(tol, Rsonic, Rmax, epsilon, dlnMdlnRInit=-1, x_high=1, x_low=1e-5, return_all_results=False):
    results = {}
    dlnMdlnRold = dlnMdlnRInit
    
    # x = v_c / 2*c_s is the iterative variable
    while x_high - x_low > tol:
        #INITIAL GUESSES
        x = 0.5 * (x_high + x_low)
        print(x)
        cs2_sonic = lf.vcgrab(Rsonic, 27.1*ktc, 210*ktc, 6.3e10, 1e12)**2 / (2 * x)
        v_sonic = cs2_sonic**0.5
        T_sonic = mu * mp_g * cs2_sonic / (gamma * k_Bcgs)
        tflow_to_tcool = (10/3) * (1 - x)
        
        rho_sonic = lf.rhocalc(v_sonic, tflow_to_tcool, T_sonic, Rsonic)
        if rho_sonic == False:
            x_high = x
            continue
        Mdot = 4 * np.pi * Rsonic**2 * rho_sonic * v_sonic
        
        dlnTdlnR1, dlnTdlnR2 = dlnTdlnrcalc(Rsonic, x, T_sonic, pr=True)
        if dlnTdlnR1 is None:
            x_high = x
            continue
        
        dlnMdlnR1, dlnMdlnR2 = [3 - 5*x - 2*dlnTdlnR for dlnTdlnR in (dlnTdlnR1, dlnTdlnR2)]
        if abs(dlnMdlnR1 - dlnMdlnRold) < abs(dlnMdlnR2 - dlnMdlnRold):
            dlnTdlnR = dlnTdlnR1
        else:
            dlnTdlnR = dlnTdlnR2
        
        dlnMdlnR = 3 - 5*x - 2*dlnTdlnR
        
        dlnvdlnR = -1.5 * dlnTdlnR + 3 - 5 * x
        
        T0 = T_sonic * (1 + epsilon * dlnTdlnR)
        v0 = v_sonic * (1 + epsilon * dlnvdlnR)
        R0 = Rsonic * (1 + epsilon)

        # Early checks
        cs2_0 = (gamma * k_Bcgs * T0) / (mu * mp_g)
        mach0 = v0 / np.sqrt(cs2_0)
        if mach0 > 1.0:
            print("starts supersonic")
            x_high = x
            continue

        phi0 = phi(R0)
        bern = 0.5 * v0**2 + 1.5 * cs2_0 + phi0
        #dvdr0, dTdr0 = TheODE(R0, [v0, T0], Mdot)
        
        if bern > 0:
            print(f"starts unbound")
            x_low = x
            continue
        res_raw = solve_ivp(TheODE, [R0, Rmax], [v0, T0], args=(Mdot,), method='RK45', max_step=R0*0.5, 
            atol=1e-8, rtol=1e-8, events=my_event_list, dense_output=True)
        
        if res_raw.status < 0:
            stop_reason = 'integration failure'
        elif any(len(evt) > 0 for evt in res_raw.t_events):
            for idx, t_evt in enumerate(res_raw.t_events):
                if len(t_evt) > 0:
                    stop_reason = event_names[idx]
                    break
        else:
            if res_raw.t[-1] >= Rmax:
                stop_reason = 'max R reached'
            else:
                stop_reason = 'unknown'
        
        res = IntegrationResult(res_raw, stop_reason)
        
        print(f"maximum r = {res.Rs()[-1] / 3.0857e21:.2f} kpc; stop reason: {res.stopReason()}")
        
        if res.stopReason() in ('sonic point', 'lowT'):
            x_high = x
            continue
        elif res.stopReason() == 'unbound':
            x_low = x
            continue
        elif res.stopReason() == 'max R reached':
            dlnMdlnRold = dlnMdlnR
            results[x] = res
            print(f"x = {x}, Tsonic = {T_sonic}, T0 = {T0} v0 = {v0}")
            res_new = solve_ivp(TheODETracker, [R0, Rmax], [v0, T0], args=(Mdot,), method='RK45', max_step=R0*0.5, 
                atol=1e-8, rtol=1e-8, dense_output=True)
            #print(res_new)
            break
        else:
            print(f"Warning: Unexpected stopReason '{res.stopReason()}' — stopping loop.")
            break
    
    if return_all_results:
        return results
    if len(results) == 0:
        print("no result reached maximum R")
        return None
    else:
        return results[x]

In [ ]:
result = sonic_point_shooting(
    tol=1e-10,
    Rsonic=19.12*ktc,
    Rmax=20600*ktc,
    epsilon=1e-5,
    return_all_results=False
)

if result is None:
    print("No solution reached max radius.")
elif isinstance(result, dict):
    for xval, res in result.items():
        print(f"x = {xval:.5f}, stop reason = {res.stopReason()}")
else:
    print(f"Stop reason: {result.stopReason()}")
    print(f"Final radius (kpc): {result.Rs()[-1] / 3.0857e21:.3f}")
    v_final, T_final = result.res.y[:, -1]
    print(f"Final velocity (cm/s): {v_final:.3e}")
    print(f"Final temperature (K): {T_final:.3e}")

In [ ]:
Tarray = np.array(Tarray)
varray = np.array(varray)
plt.scatter(ra2, varray, s=0.5, label='Velocity (v)')
plt.scatter(ra2, Tarray, s=0.5, label='Temperature (T)')
plt.xlabel('Radius (cm)')
plt.ylabel('Values')
plt.xscale('log')
plt.yscale('log')
plt.title('Various Quantities as Functions of Radius')
plt.legend()
plt.xlim(5.9e22, 3e25)
#plt.ylim(3e5, 3e7)
plt.grid(True)
#plt.savefig('dlnvdlnr components.png', transparent=True)
plt.show()

In [ ]:
rhoarray = np.array(rhoarray)
plt.scatter(ra2, rhoarray, s=0.5, label='Density (rho)')
plt.xlabel('Radius (cm)')
plt.ylabel('g/cm³')
plt.xscale('log')
plt.yscale('log')
plt.title('Various Quantities as Functions of Radius')
plt.legend()
plt.xlim(5.9e22, 3e25)
#plt.ylim(3e5, 3e7)
plt.grid(True)
#plt.savefig('dlnvdlnr components.png', transparent=True)
plt.show()

In [ ]:
vccsa2 = np.array(vc_csa)
tflowtcool = np.array(tflow_tcoola)
mach21a2 = np.array(mach2a)
dlnvdlnRa1 = np.array(dlnvdlnra)
dlnTdlnRa = np.array(dlnTdlnra)
ra2 = np.array(ra2)

dlnvdlnRa2 = dlnvdlnRa1 * -2/3
tflowtcool1 = tflowtcool * (-3/5)
tflowtcool2 = tflowtcool - (4/3)

plt.scatter(ra2, vccsa2, s=1, label='2 - v_c^2/cs^2')
plt.scatter(ra2, tflowtcool1, s=1, label='-tflow/(gamma*tcool)')
plt.scatter(ra2, mach21a2, s=1, label='Mach^2 - 1')
plt.scatter(ra2, dlnvdlnRa1, s=1, label='dlnv/dlnr')
plt.xlabel('Radius (cm)')
plt.ylabel('Values')
plt.xscale('log')
#plt.yscale('log')
plt.title('Various Quantities as Functions of Radius')
plt.legend()
plt.xlim(5.9e22, 3e25)
plt.ylim(-2, 2)
plt.grid(True)
#plt.savefig('dlnvdlnr components.png', transparent=True)
plt.show()

In [ ]:
plt.scatter(ra2, dlnvdlnRa2, s=1, label='(-2/3) * dlnv/dlnr')
plt.scatter(ra2, tflowtcool2, s=1, label='tflow/tcool - 4/3')
plt.scatter(ra2, dlnTdlnRa, s=1, label='dlnT/dlnr')
plt.xlabel('Radius (cm)')
plt.ylabel('Values')
plt.xscale('log')
#plt.yscale('log')
plt.title('Various Quantities as Functions of Radius')
plt.legend()
plt.xlim(5.9e22, 3e25)
plt.ylim(-2, 2)
plt.grid(True)
#plt.savefig('dlnvdlnr components.png', transparent=True)
plt.show()

In [ ]:
Karray = ((rhoarray/mp_g)**(-2/3)) * k_Bcgs * Tarray * etkv
#print(Karray)

plt.scatter(ra2, Karray, s=1, label='Entropy')
plt.xlabel('Radius (cm)')
plt.ylabel('K (keV*cm^2)')
plt.xscale('log')
plt.yscale('log')
plt.title('Various Quantities as Functions of Radius')
plt.legend()
plt.xlim(1.85e22, 2.16e24)
plt.ylim(10, 1000)
plt.grid(True)
#plt.savefig('dlnvdlnr components.png', transparent=True)
plt.show()